In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

import pandas as pd
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
date = '2018-05-09'

In [3]:
PTV = pd.read_csv('IPTV_'+str(date)+'_TF1.csv')[['TITRE','DUREE','description programme','HEURE',
       'debut']]
PTV['fin'] = PTV['debut']+PTV['DUREE']
Points = pd.read_csv('merged_'+str(date)+'.csv')[['minutes','heure','label','debut','fin','TITRE','DUREE','description programme', 'temps depuis debut', 'temps avant fin',
       'pourcentage déjà vu']]

Points.head(20)

In [4]:
PTV

,TITRE,DUREE,description programme,HEURE,debut,fin
0,TFou,120,Magazine jeunesse,06:25:00,385,505
1,Météo,5,Météo,08:25:00,505,510
2,Téléshopping,45,Magazine de télé-achat,08:30:00,510,555
3,Météo,5,Météo,09:15:00,555,560
4,Petits secrets entre voisins,30,Série réaliste,09:20:00,560,590
5,Petits secrets entre voisins,35,Série réaliste,09:50:00,590,625
6,Demain nous appartient,35,Série dramatique,10:25:00,625,660
7,Les feux de l'amour,55,Feuilleton sentimental,11:00:00,660,715
8,Petits plats en équilibre,5,Magazine de la gastronomie,11:55:00,715,720
9,Les douze coups de midi,55,Jeu,12:00:00,720,775


C

In [5]:
C = pd.DataFrame(PTV.iloc[[(PTV.shape[0]-1)]]).reset_index(drop=True)
C['Heure réelle'] = 0
C['minute réelle'] = 0
C['retard'] = 0
C['commentaire'] = 'début du programme'
P = PTV.shape[0]-1 #### ligne of programme we are looking at
lastend = 130
lastduree =255
current_prog = C.iloc[0]
start = True
Oups = False
for i in range(Points.shape[0]):
    if(lastend < 120):
        lastend = lastend +1440
    #take point one by one
    falsend = False
    point  = Points.iloc[i]
    verification = True
    
    while(verification):
        #Détection d'un problème dans la notation (oubli d'une fin de programme)
        if((current_prog['debut']+3+lastduree-point['minutes'])<-(min(30,max(0.5*lastduree,5)))):
            
            print("Problème: Fin de "+str(current_prog['TITRE'])+" non détectée") 
            #on écrit une ligne pour prévenir
            C = C.append(current_prog)
            C['commentaire'].iloc[(C.shape[0]-1)] = 'fin de programme non détectée'
            C['Heure réelle'].iloc[(C.shape[0]-1)] = point['heure']
            C['minute réelle'].iloc[(C.shape[0]-1)] = point['minutes']
            #Dans ce cas on Update le programme de référence
            
            if(P==(PTV.shape[0]-1) and start==True):
                P=0
                start = False
            else:
                P=P+1
            if(P>=PTV.shape[0] and start==False):
                print("Plus de programme")
                Oups = True
                break
            C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],0,'début du programme non détectée']
            current_prog = C.iloc[(C.shape[0]-1)]            
            print("Problème: début de "+str(current_prog['TITRE'])+" non détectée") 
            lastend = PTV['debut'].iloc[P]
            lastduree = PTV['DUREE'].iloc[P]
            
        else:
            verification = False
            
    if(Oups):
        break
    # if False Change Point just pass to the next one
    if (int(point['label']) == 0):
        print(str(PTV['TITRE'].iloc[P])+" faux change point")
    
    # if Pub add a ligne of pub in result
    if (int(point['label']) == 1):
        C.loc[C.shape[0]] = ['PUB',6,'publicité',point['heure'],point['minutes'],0,0,0,0,"début de publicité"]
        print(str(PTV['TITRE'].iloc[P])+" publicité")
        
    
    if (int(point['label']) == 2):
        allgood = True
        # Détection de plusiseurs points correspondant a la même fin d'un unique programme
        if(abs(lastend-point['minutes'])<3):
            print(str(PTV['TITRE'].iloc[P-1])+" detection multiple de fin de programme")
            continue
    
        #Détection de daux points (mauvaise classification)
        elif((lastend+lastduree-point['minutes'])>lastduree*0.5):
            print(str(PTV['TITRE'].iloc[P])+" detection érronée de fin de programme")
            continue
        
        else:
            verification = True
            while(verification):
                #Détection d'un problème dans la notation (oubli d'une fin de programme)
                if((current_prog['debut']+3+lastduree-point['minutes'])<-(min(30,max(0.5*lastduree,5)))):
                    print("PROBLEME") 
                    #on écrit une ligne pour prévenir
                    C = C.append(current_prog)
                    C['commentaire'].iloc[(C.shape[0]-1)] = 'fin de programme(loop) non détectée'
                    #Dans ce cas on Update le programme de référence
                    C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],0,'début du programme non détectée']
                    current_prog = C.iloc[(C.shape[0]-1)]
                    lastend = PTV['debut'].iloc[P]
                    lastduree = PTV['DUREE'].iloc[P]
                # Détection de plusiseurs points correspondant a la même fin d'un unique programme
                elif(abs(lastend-point['minutes'])<3):
                    print(str(PTV['TITRE'].iloc[P-1])+" detection(loop) multiple de fin de programme")
                    allgood = False
                    break

                #Détection de faux points (mauvaise classification)
                elif((lastend+lastduree-point['minutes'])>lastduree*0.5):
                    print(str(PTV['TITRE'].iloc[P])+" detection(loop) érronée de fin de programme")
                    allgood = False
                    break
                else:
                    verification = False
        #si tout ca se passe bien on update
        if(allgood):
            lastend = point['minutes']
            if(P>=PTV.shape[0] and start==False):
                print("Plus de programme")
                break
            
            C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],min(abs(point['minutes']-PTV['fin'].iloc[P]),abs(point['minutes']%1440-PTV['fin'].iloc[P])),"fin détectée"]
            if(P==(PTV.shape[0]-1) and start==True):
                P=0
                start = False
            else:
                P=P+1
            C.loc[C.shape[0]] = [PTV['TITRE'].iloc[P],PTV['DUREE'].iloc[P],PTV['description programme'].iloc[P],PTV['HEURE'].iloc[P],PTV['debut'].iloc[P],PTV['fin'].iloc[P],point['heure'],point['minutes'],C['retard'].iloc[(C.shape[0]-1)],'début de programme']
            current_prog = C.iloc[(C.shape[0]-1)]
            lastduree = PTV['DUREE'].iloc[P]
            C['commentaire'].iloc[(C.shape[0]-1)] = 'début de programme'
            print(str(PTV['TITRE'].iloc[P-1])+" detection de fin de programme")
        else:
            print("j'ai raté un truc")
#display_side_by_side(C,PTV)

/home/alexis/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Programmes de la nuit detection de fin de programme
TFou publicité
TFou publicité
TFou publicité
TFou detection de fin de programme
TFou detection multiple de fin de programme
Météo faux change point
Météo detection de fin de programme
Téléshopping faux change point
Téléshopping publicité
Téléshopping detection de fin de programme
Météo detection de fin de programme
Météo detection multiple de fin de programme
Petits secrets entre voisins detection de fin de programme
Petits secrets entre voisins detection de fin de programme
Demain nous appartient publicité
Demain nous appartient detection de fin de programme
Les feux de l'amour detection érronée de fin de programme
Les feux de l'amour faux change point
Les feux de l'amour publicité
Les feux de l'amour publicité
Les feux de l'amour detection de fin de programme
Les feux de l'amour detection multiple de fin de programme
Problème: Fin de Petits plats en équilibre non détectée
Problème: début de Les douze coups de midi non détectée
Les d

In [6]:
C

,TITRE,DUREE,description programme,HEURE,debut,fin,Heure réelle,minute réelle,retard,commentaire
0,Programmes de la nuit,255,Programme indéterminé,02:10:00,130,385,0,0,0,début du programme
1,Programmes de la nuit,255,Programme indéterminé,02:10:00,130,385,6h34,394,9,fin détectée
2,TFou,120,Magazine jeunesse,06:25:00,385,505,6h34,394,9,début de programme
3,PUB,6,publicité,7h17,437,0,0,0,0,début de publicité
4,PUB,6,publicité,8h5,485,0,0,0,0,début de publicité
5,PUB,6,publicité,8h20,500,0,0,0,0,début de publicité
6,TFou,120,Magazine jeunesse,06:25:00,385,505,8h24,504,1,fin détectée
7,Météo,5,Météo,08:25:00,505,510,8h24,504,1,début de programme
8,Météo,5,Météo,08:25:00,505,510,8h35,515,5,fin détectée
9,Téléshopping,45,Magazine de télé-achat,08:30:00,510,555,8h35,515,5,début de programme


In [7]:
C.to_html('newPTV.html')

In [8]:
PTV.to_html('PTV.html')

In [9]:
newPTV = C[C["description programme"] != 'publicité']

In [10]:
j = 0
for index, row in newPTV.iterrows():
        modified = False
        while(not modified):
            if(row['commentaire'] == 'début de programme'):
                print(row['TITRE'])
                if(PTV['TITRE'][j] == row['TITRE']):
                    PTV.set_value(j,'debut', row['minute réelle'])
                    modified = True
                    j = j+1
                else:
                    j = j+1
            else:
                modified = True

TFou
Météo
Téléshopping
Météo
Petits secrets entre voisins
Petits secrets entre voisins
Demain nous appartient
Les feux de l'amour
Petits plats en équilibre
L'affiche du jour
L'affiche du jour
Petits plats en équilibre
Petits plats en équilibre
La chambre maudite
La chambre maudite
La chambre maudite
Quatre mariages pour une lune de miel
Bienvenue à l'hôtel
Demain nous appartient
Météo
Le 20h le mag
Le 20h le mag
Tirage du Loto
Grey's Anatomy
Grey's Anatomy
Grey's Anatomy
Chicago Med
Chicago Med
Night Shift


/home/alexis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [11]:
PTV.to_csv('PTV_'+str(date)+'_TF1.csv',index=False)

In [12]:
print('done')

done
